# Import libraries

In [373]:
import os
import sys
import numpy as np
import pandas as pd
import cv2
import time
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
import wandb

from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from time import time
from matplotlib import image
from torchvision import transforms, datasets, models
from vit_pytorch import ViT
from vit_pytorch.distill import DistillableViT, DistillWrapper

# 완득이 Config

In [347]:
wandb.init(project='img-classification-38', 
           entity='jihoo',
           config = 
           {
               'img_size': [512, 384],
               'mean': [0.56019068, 0.52409788, 0.50145447],
               'std': [0.23318342, 0.24299835, 0.24567397],
               'learning_rate':0.001,
               'momentum': 0.9,
               'batch_size': 32,
               'num_workers': 4,
               'epoch': 7,
               'model':'resnet34'
           })
config = wandb.config


CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



In [38]:
# wandb.init(project='img-classification-38', 
#            entity='jihoo',
#            config = 
#            {
#                'img_size': [256, 192],
#                'mean': [0.5, 0.5, 0.5],
#                'std': [0.2, 0.2, 0.2],
#                'learning_rate':0.01,
#                'momentum': 0.5,
#                'batch_size': 32,
#                'num_workers': 4,
#                'epoch': 5,
#                'model':'resnet152'
#            })
# config = wandb.config

train_acc,99.79498
train_loss,0.00969
_runtime,1046
_timestamp,1630314294
_step,10
val_acc,98.86243
val_loss,0.04264


train_acc,▁▆▇███
train_loss,█▃▂▁▁▁
_runtime,▁▃▃▄▄▅▅▇▇██
_timestamp,▁▃▃▄▄▅▅▇▇██
_step,▁▂▂▃▄▅▅▆▇▇█
val_acc,▁▄▇▂█
val_loss,█▅▂█▁



CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



# Path Configurations

In [345]:
### Configurations
data_dir = 'input/data/train'
test_dir = '/opt/ml/input/data/eval'
img_dir = f'{data_dir}/images'
df_path = f'{data_dir}/csvs/new_train.csv'
model_path = f'{data_dir}/models'

# Import DataFrame

In [346]:
df = pd.read_csv(df_path)
df.head()

,Full Path,NewClass,-,Class Compare,Original Class,-.1,Mask,Gender,Age,-.2,ImageName,Gender.1,Age.1,ID
0,/opt/ml/input/data/train/images/000001_female_...,10,NaN,True,10,NaN,1,1,1,NaN,incorrect_mask.jpg,female,45,1
1,/opt/ml/input/data/train/images/000001_female_...,4,NaN,True,4,NaN,0,1,1,NaN,mask1.jpg,female,45,1
2,/opt/ml/input/data/train/images/000001_female_...,4,NaN,True,4,NaN,0,1,1,NaN,mask2.jpg,female,45,1
3,/opt/ml/input/data/train/images/000001_female_...,4,NaN,True,4,NaN,0,1,1,NaN,mask3.jpg,female,45,1
4,/opt/ml/input/data/train/images/000001_female_...,4,NaN,True,4,NaN,0,1,1,NaN,mask4.jpg,female,45,1


# Custom Dataset

In [351]:
class BaseDataset(data.Dataset):

    def __init__(self, transform=None):
        
        self.X = df['Full Path']
        self.y = df['NewClass']
        self.transform = transform

    def set_transform(self, transform):
        
        self.transform = transform

    def __getitem__(self, index):

        image = Image.open(self.X[index])
        label = self.y[index]
        
        # 이미지를 Augmentation 시킵니다.
        # image_transform = self.transform(image=np.array(image))['image']
        image_transform = self.transform(image)
        return image_transform, label

    def __len__(self):
        return len(self.X)

### Apply transform for each dataset

In [312]:
class BaseDataset(data.Dataset):

    def __init__(self, transform=None):
        
        self.X = df['Full Path']
        self.y = df['NewClass']
        self.transform = transform

    def set_transform(self, transform):
        
        self.transform = transform

    def __getitem__(self, index):

        image = Image.open(self.X[index])
        label = self.y[index]
        
        tmp_image = np.array(image)
        image_mean = tmp_image.mean(axis=(0,1)) / 255.
        image_std = tmp_image.std(axis=(0,1)) / 255.
        
        image_transform = self.transform(mean=image_mean, std=image_std)(image)
            
        return image_transform, label

    def __len__(self):
        return len(self.X)

# Apply transformation to the dataset

### functions to apply transform to each image

In [ ]:
# def get_train_transforms(mean=(0.5,0.5,0.5), std=(0.2,0.2,0.2)):
#     transforms_train = transforms.Compose([
#         transforms.Resize((img_size[0], img_size[1])),
#         transforms.RandomHorizontalFlip(), # data augmentation
#         transforms.ToTensor(),
#         transforms.Normalize(mean, std) # normalization
#     ])
#     return transforms_train

# def get_val_transforms(mean=(0.5,0.5,0.5), std=(0.2,0.2,0.2)):
#     transforms_val = transforms.Compose([
#         transforms.Resize((img_size[0], img_size[1])),
#         transforms.ToTensor(),
#         transforms.Normalize(mean, std)
#     ])
#     return transforms_val

### One single tranform function

In [352]:
def getTransformedDataset(img_size, mean, std):
    transforms_train = transforms.Compose([
        transforms.Resize((img_size[0], img_size[1])),
        transforms.RandomHorizontalFlip(), # data augmentation
        transforms.ToTensor(),
        transforms.Normalize(mean, std) # normalization
    ])

    transforms_val = transforms.Compose([
        transforms.Resize((img_size[0], img_size[1])),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    
    train_dataset.dataset.set_transform(transforms_train)
    val_dataset.dataset.set_transform(transforms_val)
    
    return train_dataset, val_dataset

# Split Dataset

In [353]:
dataset = BaseDataset()

# train dataset과 validation dataset을 8:2 비율로 나눕니다.
n_val = int(len(dataset) * 0.2)
n_train = len(dataset) - n_val
train_dataset, val_dataset = data.random_split(dataset, [n_train, n_val])

### Apply mean and std for each image

In [ ]:
# train_dataset.dataset.set_transform(get_train_transforms)
# val_dataset.dataset.set_transform(get_val_transforms)

# DataLoader

In [354]:
def getDataLoader(batch_size, num_workers):
    train_loader = data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True
    )

    val_loader = data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=False
    )
    
    return train_loader, val_loader

# Convert to GPU

In [355]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object
device

device(type='cuda', index=0)

# Model

In [356]:
def getModel(model_name, lr, momentum):
    if model_name == 'resnet152':
        model = models.resnet152(pretrained=True)
    elif model_name == 'resnet34':
        model = models.resnet34(pretrained=True)
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 18)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    
    return model, criterion, optimizer

# Train

In [357]:
def train(model, epoch, criterion, optimizer):
    num_epochs = epoch
    start_time = time()

    for epoch in range(num_epochs):
        """ Training Phase """
        model.train()

        running_loss = 0.
        running_corrects = 0

        # load a batch data of images
        for inputs, labels in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # forward inputs and get output
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # get loss value and update the network weights
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects / len(train_dataset) * 100.
        wandb.log({"train_acc":epoch_acc, "train_loss":epoch_loss})
        print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time() - start_time))

        """ Validation Phase """
        model.eval()

        with torch.no_grad():
            running_loss = 0.
            running_corrects = 0

            for inputs, labels in tqdm(val_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(val_dataset)
            epoch_acc = running_corrects / len(val_dataset) * 100.
            wandb.log({"val_acc":epoch_acc, "val_loss":epoch_loss})
            print('[Validation #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time() - start_time))

# Save Model

In [358]:
def saveModel(model, model_name):
    torch.save(model.state_dict(), os.path.join(model_path, f'{model_name}.pth'))

# Train the model with different hyperparameters

### Get the config

In [359]:
img_size = config.img_size
mean = config.mean
std = config.std
lr = config.learning_rate
momentum = config.momentum
batch_size = config.batch_size
num_workers = config.num_workers
epoch = config.epoch
model_name = config.model

config

{'img_size': [512, 384], 'mean': [0.56019068, 0.52409788, 0.50145447], 'std': [0.23318342, 0.24299835, 0.24567397], 'learning_rate': 0.001, 'momentum': 0.9, 'batch_size': 32, 'num_workers': 4, 'epoch': 7, 'model': 'resnet34'}

### Get dataset and dataloader

In [360]:
# One single mean and std
train_dataset, val_dataset = getTransformedDataset(img_size, mean, std)
train_loader, val_loader = getDataLoader(batch_size, num_workers)

In [ ]:
# separate mean and std
# train_loader, val_loader = getDataLoader(batch_size, num_workers)

### Get model

In [361]:
model, criterion, optimizer = getModel(model_name, lr, momentum)

### Train

In [362]:
train(model, epoch, criterion, optimizer)


[Train #0] Loss: 0.8742 Acc: 74.5106% Time: 112.0695s



[Validation #0] Loss: 0.3927 Acc: 87.0900% Time: 125.4185s



[Train #1] Loss: 0.2690 Acc: 91.0979% Time: 237.6095s



[Validation #1] Loss: 0.2088 Acc: 92.8042% Time: 251.0033s



[Train #2] Loss: 0.1395 Acc: 95.8201% Time: 362.9585s


...


[Validation #4] Loss: 0.0581 Acc: 98.1217% Time: 626.2615s



[Train #5] Loss: 0.0241 Acc: 99.6561% Time: 737.6450s



[Validation #5] Loss: 0.0616 Acc: 97.9894% Time: 750.9614s



[Train #6] Loss: 0.0149 Acc: 99.9074% Time: 862.7052s



[Validation #6] Loss: 0.0484 Acc: 98.4392% Time: 875.9744s


### Save the model

In [363]:
saveModel(model, 'resnet34_epoch7_simpletransform(nd)')

# Test the Model

### Libraries

In [364]:
import torchvision

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize, ToTensor, Normalize
from torchvision import transforms

### Test Dataset

In [365]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
#             tmp_image = np.array(image)
#             image_mean = tmp_image.mean(axis=(0,1)) / 255.
#             image_std = tmp_image.std(axis=(0,1)) / 255.
#             image = self.transform(mean=image_mean, std=image_std)(image)
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

### Submission csv file path and eval image path

In [366]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

### Get Test dataLoader

In [367]:
# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((img_size[0], img_size[1]), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

### Transform function for each image

In [333]:
# def get_transforms(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)):
#     transform = transforms.Compose([
#         Resize((img_size[0], img_size[1]), Image.BILINEAR),
#         ToTensor(),
#         Normalize(mean=mean, std=std),
#     ])
#     return transform

In [334]:
# dataset = TestDataset(image_paths, get_transforms)

# loader = DataLoader(
#     dataset,
#     shuffle=False
# )

### Show predicted results

In [368]:
def imshow(input, title):
    # torch.Tensor => numpy
    input = input.numpy().transpose((1, 2, 0))
    # undo image normalization
    input = std * input + mean
    input = np.clip(input, 0, 1)
    # display images
    plt.imshow(input)
    plt.title(title)
    plt.show()

In [2]:
# Test gender model
model.eval()
start_time = time()
class_names = [0, 1]

cnt = 0
with torch.no_grad():
    cnt += 1
    
    running_loss = 0.
    running_corrects = 0

    for inputs in tqdm(loader):
        inputs = inputs.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        images = torchvision.utils.make_grid(inputs)
        imshow(images.cpu(), title=preds)
            
        if cnt == 100:
            break

NameError: name 'model' is not defined

### Save predicted results

In [1]:
# combined model
model.eval()
all_predictions= []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        predicted = int(predicted.cpu().numpy())
        all_predictions.append(predicted)

NameError: name 'model' is not defined

### Save submission csv file

In [371]:
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!
